In [100]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re

In [101]:
dataset = pd.read_csv('/content/tamil_offensive_train.csv')

In [102]:
dataset.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [103]:
dataset.shape

(5880, 3)

In [104]:
dataset.drop(labels='id', axis=1, inplace=True)

In [105]:
dataset.head()

,text,category
0,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,only for விஜய் சேதுபதி and STR,NOT


In [106]:
dataset.category.unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [107]:
dataset = dataset.loc[dataset["category"] != 'not-Tamil']

In [108]:
dataset.category.unique()

array(['NOT', 'OFF'], dtype=object)

In [109]:
dataset['category'] = dataset.category.replace(to_replace=['NOT', 'OFF'], value=[0, 1])
# 0 - Not offensive
# 1 - Offensive

In [110]:
dataset.head()

,text,category
0,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,0
1,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,1
2,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,0
3,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,1
4,only for விஜய் சேதுபதி and STR,0


In [111]:
dataset.text[0]

'திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்'

In [112]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [113]:
import requests

link = "https://raw.githubusercontent.com/AshokR/TamilNLP/master/tamilnlp/Resources/TamilStopWords.txt"
req = requests.get(link)
stopwords = req.text

In [114]:
def clean_text(text):

  corpus = ''.join(str(j) for j in text)
  corpus = corpus.replace('.','')
  corpus = corpus.replace('?','')
  corpus = corpus.replace(',','')
  corpus = corpus.replace('!','')
  corpus = corpus.replace('/','')
  corpus = corpus.replace('@','')
  corpus = corpus.replace('\\','')
  corpus = corpus.replace('(','')
  corpus = corpus.replace(')','')
  corpus = corpus.replace('-','')
  corpus = corpus.replace('_','')
  corpus = corpus.replace('@','')
  corpus = corpus.replace('#','')
  corpus = corpus.replace('$','')
  corpus = corpus.replace(';','')
  corpus = corpus.replace(':','')
  corpus = corpus.replace('\'','')
  corpus = corpus.replace('\"','')
  corpus = corpus.replace('*','')
  corpus = ''.join(c for c in corpus if not c.isdigit())
  corpus = deEmojify(corpus)
  corpus = corpus.split()
  corpus = [word for word in corpus if word not in str(stopwords.split()) and not word.isalpha()]
  corpus = ' '.join(corpus)
  return corpus

In [115]:
for row in dataset:
  dataset['text'] = dataset['text'].apply(clean_text)

In [116]:
dataset.head()

,text,category
0,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,0
1,ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை தியேட்டர்...,1
2,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,0
3,மொத்த சாதியும் சாதிக்கு எதிரா நிக்குது என்னா ச...,1
4,விஜய் சேதுபதி,0


In [117]:
dataset.shape

(5877, 2)

In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import SpatialDropout1D
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.feature_selection import RFE
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [119]:
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 32000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 120
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(dataset.text.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19650 unique tokens.


In [120]:
X = tokenizer.texts_to_sequences(dataset.text.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (5877, 120)


In [121]:
Y = pd.get_dummies(dataset.category).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (5877, 2)


In [122]:
train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size=.10)

In [123]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_features, train_labels, epochs=5, batch_size=32,validation_split=0.2)
# Final evaluation of the model
model_pred_train = model.predict(train_features)
model_pred_test = model.predict(test_features)
# print(classification_report(test_labels,model_pred_test))
print('LSTM Recurrent Neural Network baseline: ' + str(roc_auc_score(train_labels, model_pred_train)))
print('LSTM Recurrent Neural Network: ' + str(roc_auc_score(test_labels, model_pred_test)))

Epoch 1/5
133/133 [==============================] - 48s 323ms/step - loss: 0.5032 - accuracy: 0.7894 - val_loss: 0.4023 - val_accuracy: 0.8261
Epoch 2/5
133/133 [==============================] - 41s 305ms/step - loss: 0.2567 - accuracy: 0.8998 - val_loss: 0.4473 - val_accuracy: 0.8138
Epoch 3/5
133/133 [==============================] - 41s 306ms/step - loss: 0.0607 - accuracy: 0.9804 - val_loss: 0.5489 - val_accuracy: 0.8166
Epoch 4/5
133/133 [==============================] - 41s 307ms/step - loss: 0.0189 - accuracy: 0.9961 - val_loss: 0.6767 - val_accuracy: 0.7996
Epoch 5/5
133/133 [==============================] - 40s 303ms/step - loss: 0.0113 - accuracy: 0.9977 - val_loss: 0.7268 - val_accuracy: 0.7873
LSTM Recurrent Neural Network baseline: 0.9846368727689551
LSTM Recurrent Neural Network: 0.8321234988795465


In [125]:
# Translation : The celebration is to love nature .. This is a new venture
news = ['இயற்கையை நேசிப்பதுதானே கொண்டாட்டம்.. இது ஒரு புது முயற்சி']
seq = tokenizer.texts_to_sequences(news)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Not Offensive','Offensive']
label = pred, labels[np.argmax(pred)]
print("News Category is: ")
print(label[1])

News Category is: 
Not Offensive


In [130]:
test_labels, model_pred_test

(array([[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]], dtype=uint8), array([[9.99920011e-01, 8.00058697e-05],
        [1.62897348e-01, 8.37102652e-01],
        [9.99895811e-01, 1.04217506e-04],
        ...,
        [9.99995947e-01, 4.00337967e-06],
        [9.99977589e-01, 2.24165124e-05],
        [9.97588515e-01, 2.41155014e-03]], dtype=float32))

In [132]:
from sklearn.metrics import confusion_matrix

model_pred_test=np.argmax(model_pred_test, axis=1)
test_labels=np.argmax(test_labels, axis=1)

confusion_matrix(test_labels, model_pred_test)


array([[413,  66],
       [ 55,  54]])

In [133]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels,model_pred_test)

0.79421768707483

In [134]:
from sklearn.metrics import classification_report
print(classification_report(test_labels,model_pred_test))

              precision    recall  f1-score   support

           0       0.88      0.86      0.87       479
           1       0.45      0.50      0.47       109

    accuracy                           0.79       588
   macro avg       0.67      0.68      0.67       588
weighted avg       0.80      0.79      0.80       588



In [135]:
from sklearn.metrics import f1_score
f1_score(test_labels,model_pred_test,average='weighted')

0.7979649116070991